<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/devops_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j openai langchain tiktoken

In [2]:
from langchain.graphs import Neo4jGraph

url = "neo4j+s://.databases.neo4j.io"
username ="neo4j"
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [3]:
import requests
import_url = "https://gist.githubusercontent.com/tomasonjo/08dc8ba0e19d592c4c3cde40dd6abcc3/raw/3d6dd149194cea8b2298c75aded4c276a9b5b42c/microservices.json"
import_query = requests.get(import_url).json()['query']
graph.query(
    import_query
)

[]

In [4]:
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.embeddings.openai import OpenAIEmbeddings

os.environ['OPENAI_API_KEY'] = "sk-"

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=url,
    username=username,
    password=password,
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
    search_type='hybrid'
)

In [5]:
vector_index.similarity_search(
    "How will recommendation service be updated?"
)

[Document(page_content='\nname: BugFix\ndescription: Address and resolve a critical bug impacting the CatalogService, affecting the user interface and experience, and hampering the overall performance and responsiveness of the service.\nstatus: Open'),
 Document(page_content='\nname: FeatureAdd\ndescription: Implement a new feature in OrderService to facilitate bulk orders, ensuring the features seamless integration with existing functionalities and maintaining the overall stability and performance of the service.\nstatus: In Progress'),
 Document(page_content='\nname: RecommendationFeature\ndescription: Add a new feature to RecommendationService to provide more personalized and accurate product recommendations to the users, leveraging user behavior and preference data.\nstatus: In Progress'),
 Document(page_content='\nname: Update\ndescription: Update InventoryService to include real-time stock updates, ensuring accurate reflection of the inventory levels and aiding in the efficient m

In [6]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

In [7]:
qa.run(
    "How will recommendation service be updated?"
)

'The recommendation service is currently in progress of being updated. The update will involve adding a new feature to the RecommendationService that will provide more personalized and accurate product recommendations to users. This update will leverage user behavior and preference data to improve the recommendations. However, the specific details of how the update will be implemented are not provided in the given context.'

In [8]:
qa.run(
    "How many open tickets there are?"
)

'There are 4 open tickets.'

In [9]:
graph.query(
    "MATCH (t:Task {status:'Open'}) RETURN count(*)")

[{'count(*)': 5}]

In [10]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, model_name='gpt-4'),
    qa_llm = ChatOpenAI(temperature=0), graph=graph, verbose=True,
)

In [11]:
chain.run(
    "How many Open tickets there are?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task {status: 'Open'}) RETURN COUNT(t)
Full Context:
[{'COUNT(t)': 5}]

> Finished chain.


'There are 5 Open tickets.'

In [12]:
chain.run(
    "Which team has the most Open tasks?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (t:Task)-[:ASSIGNED_TO]->(team:Team)
WHERE t.status = 'Open'
RETURN team.name, COUNT(t) AS OpenTasks
ORDER BY OpenTasks DESC
LIMIT 1
Full Context:
[{'team.name': 'TeamA', 'OpenTasks': 3}]

> Finished chain.


'TeamA has the most open tasks with a total of 3.'

In [13]:
chain.run(
    "Which services depend on Database directly?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON]->(d:Dependency) WHERE d.name = 'Database' RETURN m.name
Full Context:
[{'m.name': 'CatalogService'}, {'m.name': 'PaymentService'}, {'m.name': 'OrderService'}, {'m.name': 'InventoryService'}, {'m.name': 'UserService'}, {'m.name': 'AuthService'}]

> Finished chain.


'The services that depend on the Database directly are the CatalogService, PaymentService, OrderService, InventoryService, UserService, and AuthService.'

In [14]:
chain.run(
    "Which services depend on Database indirectly?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Microservice)-[:DEPENDS_ON*2..]->(d:Dependency {name: 'Database'}) RETURN m.name
Full Context:
[{'m.name': 'OrderService'}, {'m.name': 'OrderService'}, {'m.name': 'ShippingService'}, {'m.name': 'OrderService'}, {'m.name': 'UserService'}, {'m.name': 'PaymentService'}, {'m.name': 'OrderService'}]

> Finished chain.


'The services that depend on the Database indirectly are the OrderService, ShippingService, UserService, and PaymentService.'